  # Transcript Agent

*Visto a dificuldade enfrentada pelo grupo do short talks em legendar os videos e assim atrasando a publicação das entrevistas e engajamento da iniciativa, foi pensando como uma forma de facilitar a maneira de obter esse roteiro de maneira simples podendo assim editar o video com mais facilidade sem dependências externas, de inicio o agente apenas transcreve os videos/audios das entrevistas feita pleo time do ECAC short talks.*
<br>
<br>
<br>
Antes de começar, vou instalar só o que realmente preciso pra rodar esse agente.

In [ ]:
!pip install git+https://github.com/openai/whisper.git torchaudio moviepy langchain openai --upgrade

Após isso iremos importar as dependencias e em seguida montar o drive aonde estara o video para trancrição

In [ ]:
from moviepy.editor import VideoFileClip
import whisper
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

video_path = '/content/drive/MyDrive/ECAC_short_talks/entrevista_01.mp4'  # coloque aqui o vídeo real
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Not possible to find the video: {video_path}")

Configurando minha chave da OpenAI (precisa colar aqui a pessoal).

In [ ]:
os.environ["OPENAI_API_KEY"] = input("Digite sua OpenAI API Key: ")
llm = OpenAI(temperature=0.5, max_tokens=700)

Agora entram as funções principais que preparei.

In [ ]:
def extrair_audio_ecac(caminho_video):
    nome_base = os.path.splitext(os.path.basename(caminho_video))[0]
    caminho_audio = f"/content/{nome_base}.mp3"

    clip = VideoFileClip(caminho_video)
    clip.audio.write_audiofile(
        caminho_audio,
        codec="libmp3lame",
        verbose=False,
        logger=None
    )
    clip.close()

    return caminho_audio


def transcrever_ecac(caminho_audio, modelo="base"):
    modelo_whisper = whisper.load_model(modelo)
    resultado = modelo_whisper.transcribe(
        caminho_audio,
        language="pt",
        fp16=False
    )

    return resultado["text"]


Chamando as funções e salvando o roterio direto no drive

In [ ]:
arquivo_audio = extrair_audio_ecac(video_path)
texto_roteiro = transcrever_ecac(arquivo_audio)

print("==== Roteiro da Entrevista ====\n")
print(texto_roteiro)

caminho_roteiro = '/content/drive/MyDrive/your-folder/roteiro_entrevista_01.txt'
with open(caminho_roteiro, "w", encoding="utf-8") as f:
    f.write(texto_roteiro)

print(f"\nRoteiro salvo em: {caminho_roteiro}")
